In [1]:
import torch 
from train import trainAR
from models import *
from matplotlib import pyplot as plt
import build
from tqdm.notebook import tqdm
import pytorch_lightning as pl
import torch.nn.functional as F

In [2]:
import build
loader = build.dataloader(data='dataset_v6', part=0.1, batch_size=10, shuffle=True)
print(len(loader))

10


In [3]:
x, y = next(iter(loader))
print(x.shape, y.shape)

torch.Size([10, 256, 65536]) torch.Size([10])


In [4]:
print(x.min(), x.max())

tensor(0.) tensor(1.)


In [9]:
width = 10
premodel = Sequential(
    ConvBlock(256, width, 1, shift=1),
    Res(GatedConvBlock(width, width, 2)),
    ConvBlock(width, 256, 128),
)

In [10]:
class DaNet(pl.LightningModule):
    def __init__(self, model_loader, noise=0.0):
        super().__init__()
        self.save_hyperparameters()
        self.model = eval(model_loader)
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, aux = batch
        x += torch.randn_like(x) * self.hparams.noise
        x_hat = self.forward(x)
        loss = F.cross_entropy(x_hat, x)
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

In [11]:
model = DaNet(module_description(premodel), noise=0.3)
trainer = pl.Trainer(log_every_n_steps=5)
trainer.fit(model, train_dataloaders=loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 10.9 K
-------------------------------------
10.9 K    Trainable params
0         Non-trainable params
10.9 K    Total params
0.044     Total estimated model params size (MB)


Epoch 0:  10%|█         | 1/10 [00:16<02:25, 16.17s/it, loss=5.55, v_num=8]

In [ ]:
print(model)